# SED fitting with Bagpipes

While we have previously used the `EAZY` and `LePhare` classes (more details found in the [EAZY](EAZY.ipynb) and [LePhare](LePhare.ipynb) notebooks) to perform SED fitting for redshifts (and basic stellar masses/SFRs in the case of LePhare), we will now use the well-known SED fitting tool `Bagpipes` to recover galaxy properties using a Bayesian framework.

The first thing that must be done is to ensure bagpipes is downloaded and installed appropriately. At Manchester, we have slightly altered the Bagpipes basecode to include additional functionality, e.g. to directly output UV continuum slopes, $M_{\mathrm{UV}}$s, etc, as well as the option to include Gaussian redshift priors on an object by object basis. To use this functionality, you must first clone the forked bagpipes repository from Thomas Harvey's github. In an appropriate directory (i.e. outside of the galfind directory!), simply run:

git clone https://github.com/tHarvey303/bagpipes.git
cd bagpipes
pip install -e .

You will now need to ensure that the appropriate stellar, nebular, and dust emission grids are placed within the bagpipes/models/grids. Some default templates will be made available at will be made available at ...

By default, since `PyMultiNest` is not available without additional installation, bagpipes will perform the sampling using `Nautilus`, which is slower.

Should you wish to fit using `PyMultiNest`, follow the installation instructions given here:
https://johannesbuchner.github.io/PyMultiNest/

As per usual, we will first import the required modules as well as our example JOF NIRCam v11 catalogue.

In [1]:
import astropy.units as u
from copy import deepcopy
from galfind import Catalogue, EPOCHS_Selector, Bagpipes, EAZY
from galfind.Data import morgan_version_to_dir

Reading GALFIND config file from: /nvme/scratch/work/austind/GALFIND/galfind/../configs/galfind_config.ini


Failed to `import dust_attenuation`
Install from the repo with $ pip install git+https://github.com/karllark/dust_attenuation.git


In [2]:
survey = "JOF"
version = "v11"
instrument_names = ["NIRCam"]
aper_diams = [0.32] * u.arcsec
forced_phot_band = ["F277W", "F356W", "F444W"]
min_flux_pc_err = 10.

JOF_cat = Catalogue.pipeline(
    survey,
    version,
    instrument_names = instrument_names, 
    version_to_dir_dict = morgan_version_to_dir,
    aper_diams = aper_diams,
    forced_phot_band = forced_phot_band,
    min_flux_pc_err = min_flux_pc_err,
    crops = {"SELECTION": EPOCHS_Selector(allow_lowz=True). \
        _get_selection_name(aper_diams[0], \
        EAZY({"templates": "fsps_larson", "lowz_zmax": None}).label)}
)

TypeError: __init__() missing 2 required positional arguments: 'aper_diam' and 'SED_fit_label'

In [3]:
SED_fit_params_arr = [{"templates": "fsps_larson", "lowz_zmax": None}]
for SED_fit_params in SED_fit_params_arr:
    EAZY_fitter = EAZY(SED_fit_params)
    EAZY_fitter(JOF_cat, aper_diams[0], load_PDFs = False, load_SEDs = False, update = True)

NameError: name 'JOF_cat' is not defined

## Example 1: Running for a single Galaxy/Photometry_obs object

In [4]:
# TODO: More complicated than running on the full catalogue

## Example 2: Running an entire Catalogue

In [5]:
SED_fit_params = {"fix_z": EAZY_fitter().label}
pipes_fitter = Bagpipes(SED_fit_params)
pipes_fitter(JOF_cat, aper_diams[0], save_PDFs = False, load_SEDs = False, update = True)

TypeError: __call__() missing 2 required positional arguments: 'cat' and 'aper_diam'

## Example 3: Running with different priors

## Example 4: Multi-processing Catalogue SED fitting

In [6]:
# TODO: Have a look at pymultinest

## Example 4: Running with different input grids

Since Bagpipes only samples from a given set of input stellar and nebular grids created under a set of assumptions about the IMF, as well as the SPS model and CLOUDY nebular physics assumptions used, the user may wish to change these to determine whether their results are sensitive to these choices.

In [7]:
# TODO: A bit more complicated to do this